In [28]:
#from helper import *
#from llama_cpp import Llama
#from huggingface_hub import hf_hub_download
from mistral_sample import lm_mistral_generate_text



from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid

import dspy

import json
import time

import requests
from bs4 import BeautifulSoup
import markdownify
#from llama_cpp import Llama
#from llama_cpp import LogitsProcessorList
#from lmformatenforcer import CharacterLevelParser
#from lmformatenforcer.integrations.llamacpp import build_llamacpp_logits_processor
from typing import Optional
from pydantic import BaseModel, Field
from lmformatenforcer import JsonSchemaParser
from lmformatenforcer import CharacterLevelParser


In [6]:
import torch
torch.cuda.empty_cache()


### Groq model definition

In [25]:
torch.cuda.empty_cache()

llm_groq = dspy.GROQ(model='llama3-70b-8192', 
               api_key="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",
               max_tokens=8000)
#print(lm("How is India to visit in summer and what are the places to visit in summer list out 10 places?"))

In [3]:

class Conference(BaseModel):
    name: str = Field(description="Name of the conference")
    location: Optional[str] = Field(description="Location of the conference")
    date: Optional[str] = Field(description="Date of the conference in the following format DD/MM/YYYY")

class Conferences(BaseModel):
    conferences: list[Conference] = Field(description="List of Conferences")

### Pydantic model in Json format

In [4]:

class Queries(BaseModel):
    queries: list[str] = Field(description="Google Search Query for Topic")

### Function to parse the raw llm output as Json format

### Generate the search queries

### Implement chain of thoughts here so that it only respond with a dictionary without any string beforehand

In [5]:
def llm_generate_search_queries(lm_mistral_generate_text, prompt: str) -> list[str]:
    #prompt = f"{prompt}\nYou MUST answer using the following json schema: {Queries.model_json_schema()}"
    #torch.cuda.empty_cache()

    schema = Queries.model_json_schema()
    full_prompt = f"""{prompt}

You MUST respond only with a JSON object that follows this schema:
{json.dumps(schema, indent=2)}

Ensure your response is valid JSON and matches the schema exactly."""

    response = lm_mistral_generate_text(full_prompt)
    
    
    # Parse the JSON string
    parsed_json = json.loads(response)

    # Extract the queries
    queries = parsed_json["queries"]
    #torch.cuda.empty_cache()


    return queries
    
    
    

### Getting the urls from prompt

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

In [8]:
def get_top_urls_for_google_search_query(query: str, number: int = 5) -> list[dict]:
    url = f"https://www.google.com/search?q={query}"
   # torch.cuda.empty_cache()

    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    results_div = soup.find("div", id="rso") # list of urls inside this elem
    if not results_div:
        print("FAILED BECAUSE GOOGLE SEARCH OUTPUT CHANGED!")
        return []
    
    output = []
    for a_tag in results_div.find_all("a", href=True):
        title_tag = a_tag.find("h3")
        if title_tag:
            title = title_tag.get_text()
            url = a_tag['href']
            output.append({"title": title, "url": url})
            if len(output) >= number:
                break
    
    #torch.cuda.empty_cache()

    return output

### Scrapping the Google page

In [9]:
def scrape_and_convert_to_markdown(urls: list[str]) -> list[str]:
    markdown_texts = []
    for url in urls:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        try:
            markdown_text = markdownify.markdownify(str(soup))
            markdown_texts.append(markdown_text)
        except Exception as e:
            print(f"Error fetching {url}: {e}")
            continue

    return markdown_texts

### get the AI Conference details

In [32]:
def ask_llm_for_conferences(lm_mistral_generate_text, markdown_texts: list[str]) -> list[Conference]:
    import torch
    #torch.cuda.empty_cache()

    conferences = []
    
    for idx, text in enumerate(markdown_texts):
        time.sleep(60)
        print(f"Working on {idx + 1}/{len(markdown_texts)}:")
        
        schema = Conferences.model_json_schema()
        prompt = f"""You are an AI assistant. Based on the scraped search result below, extract and provide the top academic AI conferences in 2025 in the format of a JSON list.
                    Scraped Content:
                    {text[:4096]}
                    You MUST answer using the following json schema:  {json.dumps(schema, indent=2)}"""
        
        output = lm_mistral_generate_text(prompt)
        
        conferences.append(output)
    

    
            
    
    #torch.cuda.empty_cache()

    
    return conferences

### queries

In [11]:
prompt = f"""Generate five different Google search queries to find information on the top academic AI conferences in 2025.
            These queries should be tailored to discover high-impact conferences covering fields like machine learning, natural language processing,
            computer vision, and other areas within AI."""
queries = llm_generate_search_queries(lm_mistral_generate_text, prompt)
print(queries)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['Top AI conferences in 2025', 'Machine learning conferences 2025', 'Natural language processing conferences 2025', 'Computer vision conferences 2025', 'Top AI research conferences 2025']


### urls

In [12]:
urls = list({url['url'] for query in queries for url in get_top_urls_for_google_search_query(query)})
print(urls)


FAILED BECAUSE GOOGLE SEARCH OUTPUT CHANGED!
FAILED BECAUSE GOOGLE SEARCH OUTPUT CHANGED!
['https://www.datacamp.com/blog/top-ai-conferences', 'https://www.travelperk.com/blog/best-AI-conferences/', 'https://cvpr.thecvf.com/', 'https://ranlp.org/ranlp2025/', 'http://www.wikicfp.com/cfp/call?conference=computer%20vision', 'https://conferenceindex.org/conferences/computer-vision/germany', 'https://riseof.ai/conference-2025/', 'https://bigevent.io/ai-conferences/', 'https://visapp.scitevents.org/', 'http://www.wikicfp.com/cfp/call?conference=NLP', 'http://conferences.visionbib.com/Iris-Conferences.html', 'https://www.icnlp.net/', 'https://kristihines.com/ai-conferences/', 'https://www.nlpai.org/', 'https://www.aclweb.org/portal/events?order=field_event_date&sort=desc']


In [34]:
urls = list({url['url'] for query in queries for url in get_top_urls_for_google_search_query(query)})
print(urls)


['https://iotworldmagazine.com/2024/03/29/2202/list-of-top-24-ai-conferences-in-2024-and-2025', 'https://icaart.scitevents.org/', 'http://www.wikicfp.com/cfp/call?conference=NLP', 'https://internationalconferencealerts.com/blog/upcoming-computer-vision-conferences/', 'https://eventify.io/blog/ai-and-machine-learning-conferences', 'https://www.datacamp.com/blog/top-ai-conferences', 'https://www.datacamp.com/blog/top-machine-learning-conferences', 'https://ranlp.org/ranlp2025/', 'https://tsuji.tech/conferences-list-2025/', 'https://www.icnlp.net/', 'https://www.aclweb.org/portal/content/shared-tasks-conference-recent-advances-natural-language-processing-2025', 'https://aideadlin.es/', 'https://bigevent.io/ai-conferences/', 'https://www.clnlp.org/', 'https://www.travelperk.com/blog/best-AI-conferences/', 'https://tryolabs.com/blog/machine-learning-deep-learning-conferences', 'https://scholar.google.com/citations?view_op=top_venues&hl=en&vq=eng_computervisionpatternrecognition']


###  Markdowns

In [13]:
markdowns = scrape_and_convert_to_markdown(urls)

In [14]:
len(markdowns)

15

### Get conference details

In [ ]:
import torch
torch.cuda.empty_cache()
conferences  = ask_llm_for_conferences(llm_groq, markdowns)


Working on 1/15:
Working on 2/15:


In [41]:
import gc
gc.collect()

0

In [42]:
import torch
torch.cuda.empty_cache()

In [40]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()


In [18]:
x = ['After scraping the content, I extracted the top academic AI conferences in 2025 in the format of a JSON list as per the provided schema:\n\n```\n{\n  "conferences": [\n    {\n      "name": "AAAI 2025",\n      "location": "Washington, D.C., USA",\n      "date": "27/02/2025 - 03/03/2025"\n    },\n    {\n      "name": "ICML 2025",\n      "location": "Paris, France",\n      "date": "21/07/2025 - 25/07/2025"\n    },\n    {\n      "name": "IJCAI 2025",\n      "location": "Bangkok, Thailand",\n      "date": "11/07/2025 - 17/07/2025"\n    },\n    {\n      "name": "NeurIPS 2025",\n      "location": "New Orleans, Louisiana, USA",\n      "date": "06/12/2025 - 12/12/2025"\n    },\n    {\n      "name": "CVPR 2025",\n      "location": "Vancouver, British Columbia, Canada",\n      "date": "14/06/2025 - 18/06/2025"\n    },\n    {\n      "name": "ICCV 2025",\n      "location": "Seoul, South Korea",\n      "date": "08/10/2025 - 12/10/2025"\n    },\n    {\n      "name": "ECCV 2025",\n      "location": "Tel Aviv, Israel",\n      "date": "25/08/2025 - 29/08/2025"\n    },\n    {\n      "name": "ACL 2025",\n      "location": "Tokyo, Japan",\n      "date": "15/07/2025 - 20/07/2025"\n    },\n    {\n      "name": "EMNLP 2025",\n      "location": "Lisbon, Portugal",\n      "date": "10/11/2025 - 14/11/2025"\n    },\n    {\n      "name": "ICASSP 2025",\n      "location": "Rhodes, Greece",\n      "date": "18/05/2025 - 22/05/2025"\n    },\n    {\n      "name": "ICRA 2025",\n      "location": "Stockholm, Sweden",\n      "date": "03/05/2025 - 07/05/2025"\n    },\n    {\n      "name": "IJCNN 2025",\n      "location": "Rio de Janeiro, Brazil",\n      "date": "12/07/2025 - 17/07/2025"\n    },\n    {\n      "name": "ACML 2025",\n      "location": "Auckland, New Zealand",\n      "date": "04/11/2025 - 06/11/2025"\n    }\n  ]\n}\n```\n\nPlease note that the dates and locations are subject to change, and it\'s always best to verify the information with the official conference websites.']

In [20]:
print(['After scraping the content, I extracted the top academic AI conferences in 2025 in the format of a JSON list as per the provided schema:\n\n```\n{\n  "conferences": [\n    {\n      "name": "AAAI 2025",\n      "location": "Washington, D.C., USA",\n      "date": "27/02/2025 - 03/03/2025"\n    },\n    {\n      "name": "ICML 2025",\n      "location": "Paris, France",\n      "date": "21/07/2025 - 25/07/2025"\n    },\n    {\n      "name": "IJCAI 2025",\n      "location": "Bangkok, Thailand",\n      "date": "11/07/2025 - 17/07/2025"\n    },\n    {\n      "name": "NeurIPS 2025",\n      "location": "New Orleans, Louisiana, USA",\n      "date": "06/12/2025 - 12/12/2025"\n    },\n    {\n      "name": "CVPR 2025",\n      "location": "Vancouver, British Columbia, Canada",\n      "date": "14/06/2025 - 18/06/2025"\n    },\n    {\n      "name": "ICCV 2025",\n      "location": "Seoul, South Korea",\n      "date": "08/10/2025 - 12/10/2025"\n    },\n    {\n      "name": "ECCV 2025",\n      "location": "Tel Aviv, Israel",\n      "date": "25/08/2025 - 29/08/2025"\n    },\n    {\n      "name": "ACL 2025",\n      "location": "Tokyo, Japan",\n      "date": "15/07/2025 - 20/07/2025"\n    },\n    {\n      "name": "EMNLP 2025",\n      "location": "Lisbon, Portugal",\n      "date": "10/11/2025 - 14/11/2025"\n    },\n    {\n      "name": "ICASSP 2025",\n      "location": "Rhodes, Greece",\n      "date": "18/05/2025 - 22/05/2025"\n    },\n    {\n      "name": "ICRA 2025",\n      "location": "Stockholm, Sweden",\n      "date": "03/05/2025 - 07/05/2025"\n    },\n    {\n      "name": "IJCNN 2025",\n      "location": "Rio de Janeiro, Brazil",\n      "date": "12/07/2025 - 17/07/2025"\n    },\n    {\n      "name": "ACML 2025",\n      "location": "Auckland, New Zealand",\n      "date": "04/11/2025 - 06/11/2025"\n    }\n  ]\n}\n```\n\nPlease note that the dates and locations are subject to change, and it\'s always best to verify the information with the official conference websites.'])

['After scraping the content, I extracted the top academic AI conferences in 2025 in the format of a JSON list as per the provided schema:\n\n```\n{\n  "conferences": [\n    {\n      "name": "AAAI 2025",\n      "location": "Washington, D.C., USA",\n      "date": "27/02/2025 - 03/03/2025"\n    },\n    {\n      "name": "ICML 2025",\n      "location": "Paris, France",\n      "date": "21/07/2025 - 25/07/2025"\n    },\n    {\n      "name": "IJCAI 2025",\n      "location": "Bangkok, Thailand",\n      "date": "11/07/2025 - 17/07/2025"\n    },\n    {\n      "name": "NeurIPS 2025",\n      "location": "New Orleans, Louisiana, USA",\n      "date": "06/12/2025 - 12/12/2025"\n    },\n    {\n      "name": "CVPR 2025",\n      "location": "Vancouver, British Columbia, Canada",\n      "date": "14/06/2025 - 18/06/2025"\n    },\n    {\n      "name": "ICCV 2025",\n      "location": "Seoul, South Korea",\n      "date": "08/10/2025 - 12/10/2025"\n    },\n    {\n      "name": "ECCV 2025",\n      "location": 

In [23]:
print(x[0])

After scraping the content, I extracted the top academic AI conferences in 2025 in the format of a JSON list as per the provided schema:

```
{
  "conferences": [
    {
      "name": "AAAI 2025",
      "location": "Washington, D.C., USA",
      "date": "27/02/2025 - 03/03/2025"
    },
    {
      "name": "ICML 2025",
      "location": "Paris, France",
      "date": "21/07/2025 - 25/07/2025"
    },
    {
      "name": "IJCAI 2025",
      "location": "Bangkok, Thailand",
      "date": "11/07/2025 - 17/07/2025"
    },
    {
      "name": "NeurIPS 2025",
      "location": "New Orleans, Louisiana, USA",
      "date": "06/12/2025 - 12/12/2025"
    },
    {
      "name": "CVPR 2025",
      "location": "Vancouver, British Columbia, Canada",
      "date": "14/06/2025 - 18/06/2025"
    },
    {
      "name": "ICCV 2025",
      "location": "Seoul, South Korea",
      "date": "08/10/2025 - 12/10/2025"
    },
    {
      "name": "ECCV 2025",
      "location": "Tel Aviv, Israel",
      "date": "25/0

In [ ]:
client = QdrantClient(path="embeddings")
downloaded_embedding_model_path = hf_hub_download(repo_id="mixedbread-ai/mxbai-embed-large-v1", filename="gguf/mxbai-embed-large-v1-f16.gguf")
llm_embedding = Llama(model_path=downloaded_embedding_model_path, n_gpu_layers=-1, embedding=True, verbose=False)

downloaded_model_path = hf_hub_download(repo_id="bartowski/Llama-3.2-3B-Instruct-GGUF", filename="Llama-3.2-3B-Instruct-Q6_K_L.gguf")
llm = Llama(model_path=downloaded_model_path, n_ctx=16384, n_gpu_layers=-1, verbose=False)